### Image Retrieval Function

In [4]:
import requests
import json
import base64
import os
import time

headers = {
            "Content-Type": "application/json",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
          }

def seek_location(context,
                  shardId: int = 0,
                  container: str = '',
                  stream: str = "",
                  url = '',
                 secondsLapsed = 1):
    payload = {
            "ShardId": shardId,
            "Type": "TIME",
        "TimestampSec":int(time.time())-secondsLapsed,
        "TimestampNSec" : 0
          }
    headers["X-v3io-function"] = "Seek"
    r = requests.post(url, headers=headers,json=payload)
    return json.loads(r.text)['Location']

def get_image(context,
              url,
              shardId,
              container,
              stream,
              secondsLapsed):
    location = seek_location(context,shardId=shardId,
                             container=container,
                             stream=stream,
                             url=url,
                            secondsLapsed = secondsLapsed)
    img_payload = {
            "ShardId":    shardId,
            "Location":   location,
        "Limit":1
    }
    headers["X-v3io-function"] = "GetRecords"
    r = requests.post(url, headers=headers,json=img_payload)
    
    response=json.loads(r.text)
    
    return response

def handler(context, event):
    params = event.fields
    shardId=int(params['shardId'])
    container=params['container']
    stream=params['stream']
    
    url = "http://v3io-webapi:8081/%s/%s/"% (container,stream)
    headers = { "Content-Type": "application/json",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
    }

    secondsLapsed = 1
    while True:
          response = get_image(context,url,shardId,container,stream,secondsLapsed )
          data = False  
          for record in response['Records']:
                data = base64.b64decode(record['Data']) 
          if data:
            break
          else:
              secondsLapsed += 1
    
    return data